## Дескрипторы. Исключения. 

Итак, напоминаю: дескриптор - это отдельный класс, экземпляр которого пристегивается к нашему классу в качестве статического атрибута. Что делает дескриптор:

- имплементирует протокол дескриптора (Descriptor Protocol) 
- приписывается атрибуту

То есть, дескриптор может переопределять следующие методы:

    __get__(self, instance, owner)
    __set__(self, instance, value)
    __delete__(self, instance)
    __set_name__(self, owner, name)
    
Что эти методы делают и когда они вызываются:

In [37]:
class Descriptors:
  def __init__(self):
    print('__init__')
    self.name = 42
  def __set_name__(self, owner, name):
    """Вызывается при создании экземпляра класса, когда мы впервые узнаем, 
    что у нас есть атрибут с именем name, и приписываем его нашему классу"""
    print(f'__set_name__(owner={owner}, name={name})')
    self.name = name  # тут можно сразу его и задать
  def __get__(self, instance, owner=None):
    """При запрашивании атрибута у экземпляра класса"""
    print(f'__get__(instance={instance}, owner={owner})')
    return instance.__dict__.get(self.name)
  def __set__(self, instance, value):
        """При устанавливании атрибута в виде instance.name = value"""
    if self.name == 'a':
      print('a')
    instance.__dict__[self.name] = value
  def __delete__(self, instance):
    """При удалении атрибута"""
    print(f'__delete__(instance={instance})')
    del instance.__dict__[self.name]

class A:
  a = Descriptors()
  b = Descriptors()
  def __repr__(self):
    return 'A()'

class B(A):
  def __repr__(self):
    return 'B()'

__init__
__init__
__set_name__(owner=<class '__main__.A'>, name=a)
__set_name__(owner=<class '__main__.A'>, name=b)


Для чего можно использовать дескрипторы? Например, для создания атрибутов, которые нельзя перезаписывать:

In [40]:
class ReadOnlyAttribute:
  def __init__(self, value):
    self.value = value
  def __set_name__(self, owner, name):
    self.name = name
  def __get__(self, instance, owner=None):
    return self.value
  def __set__(self, instance, value):
    raise AttributeError(f'{self.name} is read-only')

class A:
  a = ReadOnlyAttribute(42)

In [41]:
ins = A()

In [42]:
ins.a

42

In [43]:
ins.a = 2

AttributeError: ignored

Также дескрипторы бывают двух видов:

- Data Descriptor (переопределяет set)
- Non Data Descriptor (переопределяет только get)

В чем разница:

In [50]:
class DataDesc:
  def __set_name__(self, owner, name):
    self.name = name
  def __get__(self, instance, owner=None):
    print('get method')
    return instance.__dict__.get(self.name)
  def __set__(self, instance, value):
    print('Set method')
    instance.__dict__[self.name] = value

class NonDataDesc:
  def __set_name__(self, owner, name):
    self.name = name
  def __get__(self, instance, owner=None):
    print('get method')
    return instance.__dict__.get(self.name)

In [51]:
class Example:
  a = DataDesc()
  b = NonDataDesc()

In [57]:
e = Example()

In [61]:
e.a

get method


1

In [64]:
e.__dict__

{'a': 1, 'b': 1}

In [62]:
e.b = 1

В момент, когда мы впервые обращаемся к атрибуту наших двух экземпляров, вызываются методы дескрипторов, потому что у обоих есть get. Но если мы попытаемся записать атрибут, то Data Descriptor вызовет свой метод, потому что у него он есть, а Non Data Descriptor потеряется и уступит место обычному протоколу, так что в дальнейшем его get перестанет работать для существующего атрибута. То есть, что будет делать питон:

Мы обратились к атрибуту по синтаксису object.attr. Питон:

- посмотрит, есть ли data дескриптор для этого атрибута;
- если нет, то будет просто проверять object.\_\_dict\_\_ на наличие такого ключа;
- если ключа там нет, только тогда проверит, а вдруг есть non data дескриптор. 

Использование дескрипторов выгодно, если у нас есть несколько атрибутов, которыми мы хотим одинаково управлять. 

Функция super() - неоднозначная; М. Лутц, например, считает, что она вообще вредная и лучше без нее. В общем и целом у нас три способа добраться до атрибутов и методов класса-родителя:

1. Напрямую через его имя: Parent.method(self, \*attrs)
2. С помощью super без аргументов: super().method(\*attrs)
3. C помощь. super с аргументами: super(Child, self).method(\*attrs)

Обратите внимание, последний вариант будет искать метод method сразу в классе Parent!

Так вот, в общем и целом использовать super() хорошо и полезно и все это делают, но когда у нас множественное наследование, мы с помощью super() иногда не можем добраться до методов того класса-родителя, который стоит в списке родителей не первым. 

Подробно про это можно почитать [здесь](https://realpython.com/python-super/). В этой статье как раз все ужасные примеры с квадратами и прочими пирамидами. 

### Исключения

Во-первых, есть такая конструкция, как try-except:

In [ ]:
a = int(input())
try:
  a += 2
except Exception:  # except (IndexError, TypeError, ValueError):
  print('String too short')
else:
  print('String long enough')
finally:
  print(a)
  print('Script ended')


Обязательные ее части - только try и except. Как она устроена:

1. try ставит маркер в том месте программы, где его вызвали, и запоминает ее состояние, а потом пытается выполнить те команды, которые написаны в его теле. 
2. Если команды внутри try вызвали ошибку, он откатывает на свой маркер и вызывает except. 
3. except смотрит, какое исключение вызвалось внутри try, и если оно указано в его аргументах, то выполняет то, что у него в теле. 
4. если ошибка не должна перехватываться except, питон ищет оператор try верхнего уровня (они бывают вложенные), а если не находит, то за работу беретс стандартный перехватчик исключений, с которым мы все постоянно имеем дело. 
5. если except внутри себя вызвал новую ошибку, происходит то же, что в пункте 4. 
6. Если ошибки в try не было, вызовется else, если он есть. 
7. Когда вся эта хитрая конструкция завершилась, вне зависимости от того, что произошло, выполнится finally. 

except может быть больше одного, а еще ему можно передавать исключения разными способами:

    except: # без всего - будет перехватывать абсолютно все исключения, включая такие, как KeyboardInterrupt, и вы не сможете остановить свою программу. 
    except Exception: # будет перехватывать исключение Exception и все исключения, которые наследуют от него
    except (TypeError, IndexError): # будет перехватывать исключения из списочка (и их детей)

Для того, чтобы искусственным образом вызвать исключение, используется оператор raise. Его тоже можно использовать несколькими способами:

    raise IndexError  # неявным образом создаст экземпляр класса IndexError и вызовет его
    raise IndexError() # сделает то же самое, но экземпляр создаем явно. В скобочках можем передать аргумент-строку. 
    exc = IndexError()
    raise exc # совсем явно создаем экземпляр класса IndexError и кладем его сперва в переменную
    raise # без всего вызывает исключение, которое возникало раньше в работе программы

Для чего может использоваться последняя форма:

In [ ]:
try:
    try:
      'ac' + 2
    except TypeError: # перехватит ошибку
      print('Type Error')
      raise # выполнит принт и все равно ее опять вызовет
except TypeError:
  print('Haha') # но мы ее снова перехватили...

Исключения можно связывать друг с другом:

In [1]:
try:
    1 / 0
except Exception as E:
    raise TypeError('Bad') from E

TypeError: Bad

Также пару слов нужно сказать про оператор assert, который обычно используется для отладки. Как он работает:

    assert <condition>[, 'message']
    
(То есть, строка-месседж не обязательная). 

Если условие в assert вернет True, то он промолчит, а если вывалится ошибка, то возникнет исключение AssertionError с тем текстом, который вы передали как 'message':

In [2]:
assert 2 > 3, 'Two is less than three you idiot!'

AssertionError: Two is less than three you idiot!

#### Кастомные исключения

Исключение в питоне - это тоже объект какого-то класса. У исключений есть своя иерархия, которая выглядит в стандартном случае как-то так:

    BaseException
    Exception
    ArithmeticError, LookupError...
    (LookupError = IndexError, KeyError...)
    
Иерархия образуется, конечно, через наследование. 

Чтобы определить собственное исключение, достаточно написать:

In [3]:
class MyExc(Exception): pass

raise MyExc('This is my exception')

MyExc: This is my exception

Можно строчку по умолчанию сразу переопределить:

In [4]:
class Career(Exception):
    def __str__(self):
        return 'So I became a linguist...'
    
raise Career

Career: So I became a linguist...

Также бывает полезно хранить в экземпляре класса своего исключения какие-то добавочные данные. 

In [5]:
class FileReadError(Exception):
    def __init__(self, line, file):
        self.line = line
        self.file = file
    def __str__(self):
        return f'Error at line {self.line}, file {self.file}'

In [6]:
try:
    raise FileReadError(42, 'myfile.txt')
except FileReadError as X:
    print(f'Error at line {X.line}, file {X.file}')

Error at line 42, file myfile.txt


Последнее, о чем мы с вами вскользь поговорили - это менеджер контекстов with. 

На самом деле поддержка диспетчер контекстов пишется самими программистами-разработчиками, и вы можете в своем приложении тоже ее реализовать для какого-то класса, но как это делать - мы рассматривать не будем, нам достаточно знать, что это такое. По духу диспетчер контекстов очень близок try-except: он точно так же запоминает состояние программы в момент своего запуска, выполняет то, что находится в его теле, и возвращается к исходному состоянию. Так, диспетчер контекстов реализован для функции open у файлов:

In [ ]:
with open(path) as file:
    file.read()

Как мы с вами прекрасно знаем, в таком случае нет необходимости явным образом закрывать файл. Закрытие файла реализовано в диспетчере контекстов. 

В дальнейшем мы с вами будем иметь дело с другими диспетчерами контекстов. Самые распространенные - это, например, диспетчер контекстов, который используется при распараллеливании процессов (если захотите, очень рекомендую поизучать библиотеку multithreading), а мы столкнемся с конструкцией

    with torch.nograd():
    
Для торча. 